# Type Specifiers

In [1]:
; (defun circular (lst)
;     (setf (cdr (last lst)) lst))

; https://stackoverflow.com/questions/30004801/common-lisp-writing-a-function-that-detects-circular-lists
(defun circular? (lst)
  "check if a list is cycles back to the first cons"
  (if (null lst)
      nil
      (loop :for cursor on (cdr lst)
              :thereis (eq cursor lst))))

(setf *print-circle* t)
(setf x '(a))
(setf (cdr x) x)
x

(circular? x)
(circular? '(a))

CIRCULAR?

T

(A)

#1=(A . #1#)

#1=(A . #1#)

T

NIL

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::X
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::X
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::X
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::X


In [2]:
(deftype proseq ()
  '(or vector (and list (not (satisfies circular?)))))

(typep #(1 2) 'proseq)

PROSEQ

T

In [ ]:
; ; The SATISFIES predicate name is not a symbol: (LAMBDA (X) (ZEROP (MOD X 4)))
; (deftype multiple-of (n)
;   `(and integer (satisfies (lambda (x)
;                              (zerop (mod x ,n))))))

; (typep 12 '(multiple-of 4))

MULTIPLE-OF

SIMPLE-TYPE-ERROR: The SATISFIES predicate name is not a symbol: (LAMBDA (X) (ZEROP (MOD X 4)))



SIMPLE-TYPE-ERROR: The SATISFIES predicate name is not a symbol: (LAMBDA (X) (ZEROP (MOD X 4)))

SIMPLE-WARNING: The SATISFIES predicate name is not a symbol: (LAMBDA (X) (ZEROP (MOD X 4)))
SB-INT:SIMPLE-STYLE-WARNING: Lisp error during constant folding:
The SATISFIES predicate name is not a symbol: (LAMBDA (X) (ZEROP (MOD X 4)))


# Binary Streams

In [6]:
(defun copy-file (from to)
  (with-open-file (in from :direction :input
                      :element-type 'unsigned-byte)
    (with-open-file (out to :direction :output
                         :element-type 'unsigned-byte
                         :if-exists :supersede)
      (do ((i (read-byte in nil -1)
              (read-byte in nil -1)))
          ((minusp i))
        (declare (fixnum i))
        (write-byte i out)))))

(copy-file "sample.txt" "sample2.txt")

COPY-FILE

NIL

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::COPY-FILE in DEFUN


# Read-Macros

In [8]:
(set-macro-character #\'
                     #'(lambda (stream char)
                         (list (quote quote) (read stream t nil t))))

T

SB-INT:SIMPLE-STYLE-WARNING: The variable CHAR is defined but never used.


In [9]:
; #!, #?, #[, #], #{, #}
(set-dispatch-macro-character
  #\# #\?
  #'(lambda (stream char1 char2)
      (list 'quote
            (let ((lst nil))
              (dotimes (i (+ (read stream t nil t)) 1)
                (push i lst))
              (nreverse lst)))))

#?7

T

(0 1 2 3 4 5 6)

SB-INT:SIMPLE-STYLE-WARNING: The variable CHAR1 is defined but never used.
SB-INT:SIMPLE-STYLE-WARNING: The variable CHAR2 is defined but never used.


In [11]:
(set-macro-character #\} (get-macro-character #\)))

(set-dispatch-macro-character
  #\# #\{
  #'(lambda (stream char1 char2)
      (let ((accum nil)
            (pair (read-delimited-list #\} stream t)))
        (do ((i (car pair) (+ i 1)))
            ((> i (cadr pair))
             (list 'quote (nreverse accum)))
          (push i accum)))))

#{2 7}

T

T

(2 3 4 5 6 7)

SB-INT:SIMPLE-STYLE-WARNING: The variable CHAR1 is defined but never used.
SB-INT:SIMPLE-STYLE-WARNING: The variable CHAR2 is defined but never used.


# Packages

In [12]:
(package-name *package*)

"COMMON-LISP-USER"

In [13]:
(find-package 'cl-user)

#<PACKAGE "COMMON-LISP-USER">

In [ ]:
; find the package where a symbol is interned
(symbol-package 'sym)

#<PACKAGE "COMMON-LISP-USER">

In [ ]:
(setf sym 99)
(setf *package* (make-package 'mine :use '(cl)))
; sym ; UNBOUND-VARIABLE: The variable SYM is unbound.

99

#<PACKAGE "MINE">

UNBOUND-VARIABLE: The variable SYM is unbound.



UNBOUND-VARIABLE: The variable SYM is unbound.

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::SYM
SIMPLE-WARNING: undefined variable: MINE::SYM


In [17]:
cl-user::sym

99

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::SYM


In [18]:
(in-package cl-user)
(export 'bar)
(setf bar 5)

(in-package mine)
cl-user:bar

#<PACKAGE "COMMON-LISP-USER">

T

5

#<PACKAGE "MINE">

5

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER:BAR
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER:BAR


In [19]:
(import 'cl-user:bar)
bar

T

5

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER:BAR


In [ ]:
; (import 'cl-user::sym)

NAME-CONFLICT: IMPORT (COMMON-LISP-USER::SYM) causes name-conflicts in #<PACKAGE "MINE">
between the following symbols:
  MINE::SYM, COMMON-LISP-USER::SYM
See also:
  The ANSI Standard, Section 11.1.1.2.5



NAME-CONFLICT: IMPORT (COMMON-LISP-USER::SYM) causes name-conflicts in #<PACKAGE "MINE">
between the following symbols:
  MINE::SYM, COMMON-LISP-USER::SYM
See also:
  The ANSI Standard, Section 11.1.1.2.5

In [21]:
*package*

#<PACKAGE "MINE">

In [22]:
(use-package 'cl-user)
#'cons

T

#<FUNCTION CONS>

In [24]:
(in-package cl-user)

#<PACKAGE "COMMON-LISP-USER">

# The Loop Facility

In [25]:
(loop for x from 0 to 9
      do (princ x))

NIL

0123456789

In [26]:
(loop for x = 8 then (/ x 2)
      until (< x 1)
      do (princ x))

NIL

8421

In [27]:
(loop for x from 1 to 4
        and y from 1 to 4
      do (princ (list x y)))

NIL

(1 1)(2 2)(3 3)(4 4)

In [28]:
(loop for x in '(1 2 3 4)
      collect (1+ x))

(2 3 4 5)

In [29]:
(defun my-sum (n)
  (loop for x from 1 to n sum x))
(my-sum 100)

MY-SUM

5050

In [34]:
(funcall #'length "abc")

3

In [2]:
(defun most (fn lst)
  (if (null lst)
      (values nil nil)
      (let* ((wins (car lst))
             (max (funcall fn wins)))
        (dolist (obj (cdr lst))
          (let ((score (funcall fn obj)))
            (when (> score max)
                  (setf wins obj
                    max score))))
        (values wins max))))

(defun most-loop (fn lst)
  (if (null lst)
      (values nil nil)
      (loop with wins = (car lst)
            with max = (funcall fn wins)
            for obj in (cdr lst)
            for score = (funcall fn obj)
              when (> score max)
            do (setf wins obj
                 max score)
            finally (return (values wins max)))))

(let ((lst '("abc" "d" "ef")))
  (write (most #'length lst))
  (write (most-loop #'length lst)))

MOST

MOST-LOOP

"abc"

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::MOST in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::MOST-LOOP in DEFUN
"abc""abc"

# Conditions

In [ ]:
; (error "Your report use ~A as a verb." 'status)

SIMPLE-ERROR: Your report use STATUS as a verb.



SIMPLE-ERROR: Your report use STATUS as a verb.

In [ ]:
; (ecase 1 (2 3) (4 5))

CASE-FAILURE: 1 fell through ECASE expression. Wanted one of (2 4).



CASE-FAILURE: 1 fell through ECASE expression. Wanted one of (2 4).

In [ ]:
; (let ((x '(a b c)))
;   (check-type (car x) integer "an integer")
;   x)

SIMPLE-TYPE-ERROR: The value of (CAR X) is A, which is not an integer.



SIMPLE-TYPE-ERROR: The value of (CAR X) is A, which is not an integer.

SB-INT:CONSTANT-MODIFIED: Destructive function SB-KERNEL:%RPLACA called on constant data: (A B C)
See also:
  The ANSI Standard, Special Operator QUOTE
  The ANSI Standard, Section 3.7.1


In [ ]:
; (let ((sandwich '(ham on rye)))
;   (assert (eql (car sandwich) 'chicken)
;       ((car sandwich))
;       "I wanted a ~A sandwich." 'chicken)
;   sandwich)

SIMPLE-ERROR: I wanted a CHICKEN sandwich.



SIMPLE-ERROR: I wanted a CHICKEN sandwich.

SB-INT:CONSTANT-MODIFIED: Destructive function SB-KERNEL:%RPLACA called on constant data: (HAM ON RYE)
See also:
  The ANSI Standard, Special Operator QUOTE
  The ANSI Standard, Section 3.7.1


In [ ]:
(defun user-input (prompt)
  (format t prompt)
  (let ((str (read-line)))
    (format t "~A~%" str)
    (or (ignore-errors (read-from-string str))
        nil)))

(user-input "please type an expression> ") ; #%@#+!!

USER-INPUT

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::USER-INPUT in DEFUN
please type an expression> 

NIL

#%@#+!!
